# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

- Таблица содержит 12 столбцов и 21525 строк c типом данных в скобках:
     - children — количество детей в семье
     - days_employed — общий трудовой стаж в днях
     - dob_years — возраст клиента в годах
     - education — уровень образования клиента
     - education_id — идентификатор уровня образования
     - family_status — семейное положение
     - family_status_id — идентификатор семейного положения
     - gender — пол клиента
     - income_type — тип занятости
     - debt — имел ли задолженность по возврату кредитов
     - total_income — ежемесячный доход
     - purpose — цель получения кредита

## Шаг 1. Откроем файл с данными и изучим общую информацию


In [ ]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()

data = pd.read_csv('/datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**



В столбце days_employed встречаются отрицательные значени. Также в этом столбце есть пропущенные значения NaN. Еще NaN встречается в столбце total_income. Их нужно заполнить и понять причину возникновения. total_income округлим и переведем в int, для удобства. Приведем education к общему регистру. Максимальное и минимальной количество детей, возможно опечатки. В столбце days_employed стаж у пенсионеров > 1000 лет. 

## Шаг 2. Предобработка данных

### Обработка пропусков

In [ ]:
data[data['days_employed'].isna()].count() # узнаем количество пропущенных значений

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

In [ ]:
total_income_group_mean = data.groupby('income_type')['total_income'].transform('mean') # средний доход по типу занятости  
data['total_income'].fillna(total_income_group_mean, inplace=True) # заменяем NaN в 'total_income' на средний доход по типу занятости 

data['days_employed'] = data['days_employed'].apply(abs) # избавляемся от отрицательных значений в 'days_employed'

days_employed_group_mean = data.groupby('income_type')['days_employed'].transform('mean') # среднее 'days_employed' по типу занятости  
data['days_employed'].fillna(days_employed_group_mean, inplace=True)
data.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [ ]:
dob_years_mean = data['dob_years'].mean() # средний возраст
data['dob_years'] = data['dob_years'].replace(0, int(dob_years_mean)) # заменим 0 на средний возраст


data['children'] = data['children'].apply(abs) # переведем отрицательное значение 'children' в положительное
data['children'] = data['children'].replace(20, 2) # заменим количество детей с 20 на 2

data.info()
data.tail(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


**Вывод**

Заменили пропущенные значения по доходу на среднее для каждой категории занятости (Я думаю так будет точнее). Избавились от отрицательных значений в трудовом стаже, заполнили пропуски средним значение по категории занятости. Возможно клиенты не указывали свой возравст и доход. У пенсионеров получился средний стаж 365000 дней. Этот столбец для решения задачи нам не пригодится, но об ошибке необходимо предупредить. Знасения 0 в столбце возраст заменил на среднее по столбцу (хотел заменить на среднее по категории занятости, но, почемуто не получилось. Код закоментировал, может подскажете, где ошибка). Исправили количество детей с 20 на 2 (скорее всего опечатка).

### Замена типа данных

In [ ]:
data['total_income'] = data['total_income'].astype('int') # преводим значения 'total_income' в челое int для удобства
data['dob_years'] = data['dob_years'].astype('int') # преводим значения 'dob_years' в int для удобства
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


**Вывод**

Преводим значения 'dob_years' и 'total_income' в int для удобства

### Обработка дубликатов

In [ ]:
data['education'] = data['education'].str.lower() # привели 'education' к нижнему регистру
data.duplicated().sum() # посмотрим на суммарное количество дубликатов



71

In [ ]:
data[data.duplicated(keep=False)].sort_values(by=['total_income', 'dob_years']) # посмотрим на дубликаты

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
17787,0,365003.491245,54,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции с жильем
21415,0,365003.491245,54,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции с жильем
3344,0,365003.491245,56,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции со своей недвижимостью
9627,0,365003.491245,56,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции со своей недвижимостью
13300,0,365003.491245,56,среднее,1,женат / замужем,0,F,пенсионер,0,137127,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
19369,0,2111.524398,45,среднее,1,гражданский брак,1,F,компаньон,0,202417,свадьба
9920,0,2111.524398,51,среднее,1,гражданский брак,1,F,компаньон,0,202417,на проведение свадьбы
15991,0,2111.524398,51,среднее,1,гражданский брак,1,F,компаньон,0,202417,на проведение свадьбы
2254,0,2111.524398,54,высшее,0,женат / замужем,0,M,компаньон,0,202417,операции с коммерческой недвижимостью


In [ ]:
data = data.drop_duplicates().reset_index() # удаляем дубликаты

In [ ]:
data.duplicated().sum() # проверяем еще раз дубликаты

0

**Вывод**

Привели 'education' к нижнему регистру. Выявили и удалили дубликаты. Причина появления дублей, скорее всего, в работе скрипта. Человек несколько раз нажимал на отправку формы и форма отправлялась несколько раз.

### Лемматизация

In [ ]:
data['purpose'].value_counts() # посмотрим на все цели кредита

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [ ]:
from pymystem3 import Mystem
m = Mystem()
def purpose_lemmas(purpose):
     lemmas = m.lemmatize(purpose)  # лемматизировали цели кредита
     return lemmas                
data['purpose_lemmas'] = data['purpose'].apply(purpose_lemmas)
data.head(10)

,index,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
0,0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"
5,5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]"
6,6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]"
7,7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]"
8,8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]"
9,9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]"


In [ ]:
data['purpose_lemmas'].value_counts() # выделим топ целей (автомобиль, свадьба, недвижимость, образование)

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 4588, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

**Вывод**

Лемматизацию записал в новый столбец для дальнейшей категоризации. Перед лемматизацией вывел цели для ознакомления. После лемматизации веделил основные категории (автомобиль, свадьба, недвижимость, образование).

### Категоризация данных

In [ ]:
def purpose_cat(purpose):    # присвоили категории всем клиентам по целям
    if 'автомобиль'in purpose:
        return 'авто'
    elif 'свадьба' in purpose:
        return 'свадьба'
    elif 'недвижимость' in purpose or 'жилье' in purpose:
        return 'недвижимость'
    elif 'образование' in purpose:
        return 'образование'
    else:
        return 'другое'
data['purpose_cat'] = data['purpose_lemmas'].apply(purpose_cat)
data['purpose_cat'].value_counts()


недвижимость    10811
авто             4306
образование      4013
свадьба          2324
Name: purpose_cat, dtype: int64

In [ ]:
data['dob_years'].sort_values() # смотрим возраст заемщиков

5558     19
12998    19
9208     19
20166    19
2725     19
         ..
4891     74
12297    74
3458     74
2557     74
8870     75
Name: dob_years, Length: 21454, dtype: int64

In [ ]:
def dob_years_group(age): # категории по возрасту
    if 18 <= age <= 35:
        return 'молодые'
    if 36 <= age <= 65:
        return 'средний возраст'
    if age > 65:
        return 'пенсионеры'
dob_years_group(66)

'пенсионеры'

In [ ]:
data['dob_years_group'] = data['dob_years'].apply(dob_years_group)
data.head(10)

,index,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,purpose_cat,dob_years_group
0,0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,средний возраст
1,1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",авто,средний возраст
2,2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,молодые
3,3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,молодые
4,4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,средний возраст
5,5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",недвижимость,молодые
6,6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость,средний возраст
7,7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",образование,средний возраст
8,8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба,молодые
9,9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",недвижимость,средний возраст


In [ ]:
data['total_income'].sort_values() # смотрим разброс доходов

14555      20667
12983      21205
16138      21367
1598       21695
14247      21895
          ...   
17137    1711309
20742    1715018
9159     1726276
19548    2200852
12390    2265604
Name: total_income, Length: 21454, dtype: int64

In [ ]:
def total_income_group(income): # разбиваем на категории по доходу
    if income <= 60000:
        return 'доход низкий'
    elif income <= 120000:
        return 'доход средний'
    elif income <= 180000:
        return 'доход выше среднего'
    elif income > 180000:
        return 'доход высокий'
data['total_income_group'] = data['total_income'].apply(total_income_group)
data.head(10)

,index,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,purpose_cat,dob_years_group,total_income_group
0,0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,средний возраст,доход высокий
1,1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",авто,средний возраст,доход средний
2,2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,молодые,доход выше среднего
3,3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,молодые,доход высокий
4,4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,средний возраст,доход выше среднего
5,5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",недвижимость,молодые,доход высокий
6,6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость,средний возраст,доход высокий
7,7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",образование,средний возраст,доход выше среднего
8,8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба,молодые,доход средний
9,9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",недвижимость,средний возраст,доход выше среднего


**Вывод**

Произвели категоризацию данных по целям, возрасту и уровню дохода. Категории по возрасту и доходу искал в сети, данные разнятся, поэтому разбил на такие категории.

In [ ]:
education_cat = data.groupby(['education', 'education_id']).size().to_frame('count')
education_cat.sort_values(by='education_id')

,,count
education,education_id,
высшее,0,5250
среднее,1,15172
неоконченное высшее,2,744
начальное,3,282
ученая степень,4,6


In [ ]:
family_status_cat = data.groupby(['family_status', 'family_status_id']).size().to_frame('count')
family_status_cat.sort_values(by='family_status_id')

,,count
family_status,family_status_id,
женат / замужем,0,12339
гражданский брак,1,4151
вдовец / вдова,2,959
в разводе,3,1195
Не женат / не замужем,4,2810


## Шаг 3. Ответим на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
data_pivot_children_debt = data.pivot_table(index = ['children'], values = 'debt').round(3) # сводная таблица по детям
data_pivot_children_debt.sort_values(by='debt')

,debt
children,
5,0.000
0,0.075
3,0.082
1,0.092
2,0.095
4,0.098


**Вывод**

Из таблицы видно, что заемщики с количеством детей от 1 до 4 возвращают долг реже, чем заемщики без детей и с 5 детьми.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
data_pivot_family_debt = data.pivot_table(index = ['family_status'], columns = 'debt', values = 'gender', aggfunc = 'count')
data_pivot_family_debt['family_debt'] = round(data_pivot_family_debt[1] / (data_pivot_family_debt[0] + data_pivot_family_debt[1]), 3)
data_pivot_family_debt.sort_values(by='family_debt') # сводная таблица по семейному положению

debt,0,1,family_debt
family_status,,,
вдовец / вдова,896,63,0.066
в разводе,1110,85,0.071
женат / замужем,11408,931,0.075
гражданский брак,3763,388,0.093
Не женат / не замужем,2536,274,0.098


**Вывод**

Заемщики не состоящие в официальных отношениях (Не женат / не замужем, гражданский брак) возвращают долг реже, чем остальные.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
data_pivot_income_debt = data.pivot_table(index = ['total_income_group'], columns = 'debt', values = 'gender', aggfunc = 'count')
data_pivot_income_debt['income_debt'] = round(data_pivot_income_debt[1] / (data_pivot_income_debt[0] + data_pivot_income_debt[1]), 3)
data_pivot_income_debt.sort_values(by='income_debt') # сводная таблица по уровню дохода

debt,0,1,income_debt
total_income_group,,,
доход низкий,757,49,0.061
доход высокий,6416,508,0.073
доход средний,5537,502,0.083
доход выше среднего,7003,682,0.089


**Вывод**

Люди с низким доходом чаще стараются возвращать кредит в срок. Реже возвращают люди с высоким доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
data_pivot_purpose_debt = data.pivot_table(index = ['purpose_cat'], columns = 'debt', values = 'gender', aggfunc = 'count')
data_pivot_purpose_debt['purpose_debt'] = round(data_pivot_purpose_debt[1] / (data_pivot_purpose_debt[0] + data_pivot_purpose_debt[1]), 3)
data_pivot_purpose_debt.sort_values(by='purpose_debt') # сводная таблица по цели кредита

debt,0,1,purpose_debt
purpose_cat,,,
недвижимость,10029,782,0.072
свадьба,2138,186,0.080
образование,3643,370,0.092
авто,3903,403,0.094


**Вывод**

Люди, берущие кредит на недвижимость, возвращают долг чаще, чем люди, берущие кредит, на образование или авто.

## Шаг 4. Общий вывод

Идеальный заемщик: человек с 5-ю детьми, овдовевший, с низким уровнем дохода, берущий кредит на недвижимость.

<div>
<font size="3"><b>Вывод</b></font>
    
В ходе исследования мы изучили датафрейм, выявили ошибки и пропуски. Исправили и заполнили средними значениями по категориям. 
    
Выявили и удалили дубликаты. Произвели лемматизацию и категоризацию данных.
    
В ходе исследования выявили зависимость между наличием детей и возвратом кредита в срок. 
Заемщики с 5-ю детьми не имеют задолжностей. Заемщики без детей возвращаю кредиты чаще, чем заемщики с 1-4 детьми.
    
Также выяснили, что овдовевшие заемщики платят чаще, чем люди не имеющие официальных отношений.
    
Выявили зависимость между уровнем дохода и возвратом кредита в срок. Заемщики с низким и высоким доходом более ответственные, 
чем заемщики со средним доходом и доходом выше среднего.
    
Также на возват кредита в срок влияют и цели кредита. Кредит на недвижимость и свадьбу возвращают чаще, чем кредит на 
    образование и автомобиль.
</div>